In [1]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 11.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    roc_auc_score,
    average_precision_score,
    classification_report
)
from sklearn.preprocessing import label_binarize

## 데이터 불러오기

In [3]:
train = pd.read_csv("/content/drive/MyDrive/Coursework/25-2 Machine Learning/Project/DataPreprocessing_final/openworld_train.csv")
test = pd.read_csv("/content/drive/MyDrive/Coursework/25-2 Machine Learning/Project/DataPreprocessing_final/openworld_test.csv")

y_train = train['label']
X_train = train.drop(columns=['label'])

y_test = test['label']
X_test = test.drop(columns=['label'])

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (20300, 26)
y_train shape: (20300,)
X_test shape: (8700, 26)
y_test shape: (8700,)


In [4]:
NEG_LABEL = 95
RANDOM_STATE = 42
TEST_SIZE = 0.2  # validation 비율
EARLY_STOP = 100

X_tr, X_val, y_tr_full, y_val_full = train_test_split(
    X_train, y_train, test_size=TEST_SIZE, stratify=y_train, random_state=RANDOM_STATE
)

# 1단계 이진 라벨 (양성=1: NEG_LABEL이 아닌 모든 클래스)
y_tr_bin = (y_tr_full != NEG_LABEL).astype(int)
y_val_bin = (y_val_full != NEG_LABEL).astype(int)
y_test_bin = (y_test != NEG_LABEL).astype(int)

## 1단계-이진 분류 및 성능 확인

In [7]:
#xgb
xgb_best_params_bin = {'bin_lr': 0.1890408241426905, 'bin_max_depth': 10, 'bin_min_child_weight': 1, 'bin_subsample': 0.8887320375513359, 'bin_colsample_bytree': 0.9912853863634461, 'bin_gamma': 0.014498503088113818, 'bin_reg_lambda': 2.7498279931612504, 'bin_reg_alpha': 0.0985171472007404}
xgb_params_bin = {
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "n_estimators": 1000,
    "tree_method": "hist",
    "random_state": RANDOM_STATE,
    "early_stopping_rounds": 500,
    **xgb_best_params_bin
}

xgb_model_bin = XGBClassifier(**xgb_params_bin)
xgb_model_bin.fit(
    X_tr, y_tr_bin,
    eval_set=[(X_tr, y_tr_bin), (X_val, y_val_bin)],
    verbose=1
)

#catboost
cat_best_params_bin = {'depth': 7, 'learning_rate': 0.10232460060620925, 'l2_leaf_reg': 7.533515426628427, 'border_count': 195, 'bagging_temperature': 1.0674918075757591, 'random_strength': 2.334608775118328, 'iterations': 1397}
cat_params_bin = {
    "loss_function": "Logloss",
    "eval_metric": "Logloss",
    "random_state": RANDOM_STATE,
    **cat_best_params_bin

}
cat_model_bin = CatBoostClassifier(**cat_params_bin)
cat_model_bin.fit(X_tr, y_tr_bin)

#ensemble 결과
w_xgb = 0.4
w_cat = 0.6

xgb_proba = xgb_model_bin.predict_proba(X_test)
cat_proba = cat_model_bin.predict_proba(X_test)

ens_proba = w_xgb * xgb_proba + w_cat * cat_proba
ens_pred_bin = np.argmax(ens_proba, axis=1)

[0]	validation_0-logloss:0.54317	validation_1-logloss:0.54307
[1]	validation_0-logloss:0.48503	validation_1-logloss:0.48523
[2]	validation_0-logloss:0.43779	validation_1-logloss:0.44265
[3]	validation_0-logloss:0.40984	validation_1-logloss:0.41812
[4]	validation_0-logloss:0.38853	validation_1-logloss:0.39914
[5]	validation_0-logloss:0.37340	validation_1-logloss:0.38560
[6]	validation_0-logloss:0.36062	validation_1-logloss:0.37594
[7]	validation_0-logloss:0.35196	validation_1-logloss:0.36828
[8]	validation_0-logloss:0.34033	validation_1-logloss:0.35944
[9]	validation_0-logloss:0.32906	validation_1-logloss:0.35054
[10]	validation_0-logloss:0.32408	validation_1-logloss:0.34676
[11]	validation_0-logloss:0.31515	validation_1-logloss:0.33904
[12]	validation_0-logloss:0.30938	validation_1-logloss:0.33470
[13]	validation_0-logloss:0.30428	validation_1-logloss:0.33064
[14]	validation_0-logloss:0.29590	validation_1-logloss:0.32673
[15]	validation_0-logloss:0.29289	validation_1-logloss:0.32354
[1

/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [12:41:24] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "bin_colsample_bytree", "bin_gamma", "bin_lr", "bin_max_depth", "bin_min_child_weight", "bin_reg_alpha", "bin_reg_lambda", "bin_subsample" } are not used.

  self.starting_round = model.num_boosted_rounds()


[19]	validation_0-logloss:0.26914	validation_1-logloss:0.30817
[20]	validation_0-logloss:0.26446	validation_1-logloss:0.30615
[21]	validation_0-logloss:0.26027	validation_1-logloss:0.30427
[22]	validation_0-logloss:0.25769	validation_1-logloss:0.30313
[23]	validation_0-logloss:0.25637	validation_1-logloss:0.30202
[24]	validation_0-logloss:0.25038	validation_1-logloss:0.29844
[25]	validation_0-logloss:0.24926	validation_1-logloss:0.29757
[26]	validation_0-logloss:0.24677	validation_1-logloss:0.29672
[27]	validation_0-logloss:0.24500	validation_1-logloss:0.29561
[28]	validation_0-logloss:0.24268	validation_1-logloss:0.29473
[29]	validation_0-logloss:0.24071	validation_1-logloss:0.29338
[30]	validation_0-logloss:0.23817	validation_1-logloss:0.29180
[31]	validation_0-logloss:0.23535	validation_1-logloss:0.29076
[32]	validation_0-logloss:0.23091	validation_1-logloss:0.28793
[33]	validation_0-logloss:0.22654	validation_1-logloss:0.28567
[34]	validation_0-logloss:0.22287	validation_1-logloss:

In [8]:
#ensemble 결과
w_xgb = 0.4
w_cat = 0.6

xgb_proba = xgb_model_bin.predict_proba(X_test)
cat_proba = cat_model_bin.predict_proba(X_test)

ens_proba = w_xgb * xgb_proba + w_cat * cat_proba
ens_pred_bin = np.argmax(ens_proba, axis=1)

In [9]:
stage1_acc_top1 = accuracy_score(y_test_bin, ens_pred_bin)
stage1_f1_micro = f1_score(y_test_bin, ens_pred_bin, average="micro")
stage1_f1_macro = f1_score(y_test_bin, ens_pred_bin, average="macro")
stage1_f1_weighted = f1_score(y_test_bin, ens_pred_bin, average="weighted")

stage1_auc  = roc_auc_score(y_test_bin, ens_proba[:,1])
stage1_aupr = average_precision_score(y_test_bin, ens_proba[:,1])

print("Binary_acc_top1:", stage1_acc_top1)
print("Binary_f1_micro:", stage1_f1_micro)
print("Binary_f1_macro:", stage1_f1_macro)
print("Binary_f1_weighted:", stage1_f1_weighted)
print("Binary_auc:", stage1_auc)
print("Binary_aupr:", stage1_aupr)

Binary_acc_top1: 0.9041379310344828
Binary_f1_micro: 0.9041379310344828
Binary_f1_macro: 0.8914924830751423
Binary_f1_weighted: 0.9029883448563609
Binary_auc: 0.9636836257309943
Binary_aupr: 0.979859456125102


## 2단계 - 다중 분류 및 성능 확인

### 개별 모델 학습

In [5]:
tr_pos_mask = (y_tr_full != NEG_LABEL)

# 폴드(여기선 트레인) 양성 라벨을 0..C-1로 재인코딩
pos_le = LabelEncoder().fit(y_tr_full[tr_pos_mask])
y_tr_pos_enc = pos_le.transform(y_tr_full[tr_pos_mask])
num_pos_classes = len(pos_le.classes_)

In [6]:
#xgb
xgb_best_params = {'learning_rate': 0.19721925924769965, 'max_depth': 10, 'min_child_weight': 4, 'subsample': 0.7577072390567685, 'colsample_bytree': 0.6494394276320198, 'gamma': 0.06948732401595321, 'reg_lambda': 3.2137571796403774, 'reg_alpha': 0.446034681615163}
xgb_params = {
    "objective": "multi:softprob",
    "num_class": 95,
    "eval_metric": "mlogloss",
    "n_estimators": 10000,
    "random_state": RANDOM_STATE,
    "early_stopping_rounds":5000,
    **xgb_best_params
}

xgb = XGBClassifier(**xgb_params)
xgb.fit(
    X_tr[tr_pos_mask], y_tr_pos_enc,
    eval_set=[(X_tr[tr_pos_mask], y_tr_pos_enc)],
    verbose=1
)

#catboost
cat = CatBoostClassifier(
    loss_function="MultiClass",
    eval_metric="MultiClass",
    random_seed=RANDOM_STATE
)

cat.fit(X_tr[tr_pos_mask], y_tr_pos_enc,
    eval_set=[(X_tr[tr_pos_mask], y_tr_pos_enc)])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[6005]	validation_0-mlogloss:0.07077
[6006]	validation_0-mlogloss:0.07077
[6007]	validation_0-mlogloss:0.07077
[6008]	validation_0-mlogloss:0.07077
[6009]	validation_0-mlogloss:0.07077
[6010]	validation_0-mlogloss:0.07077
[6011]	validation_0-mlogloss:0.07077
[6012]	validation_0-mlogloss:0.07077
[6013]	validation_0-mlogloss:0.07076
[6014]	validation_0-mlogloss:0.07076
[6015]	validation_0-mlogloss:0.07076
[6016]	validation_0-mlogloss:0.07076
[6017]	validation_0-mlogloss:0.07076
[6018]	validation_0-mlogloss:0.07076
[6019]	validation_0-mlogloss:0.07075
[6020]	validation_0-mlogloss:0.07075
[6021]	validation_0-mlogloss:0.07075
[6022]	validation_0-mlogloss:0.07075
[6023]	validation_0-mlogloss:0.07075
[6024]	validation_0-mlogloss:0.07075
[6025]	validation_0-mlogloss:0.07075
[6026]	validation_0-mlogloss:0.07075
[6027]	validation_0-mlogloss:0.07075
[6028]	validation_0-mlogloss:0.07075
[6029]	validation_0-mlogloss:0.07075
[6030]	validation_0-mlogloss:0.07075
[6

In [10]:
# 1단계에서 positive로 예측된 인덱스
cond_idx = (ens_pred_bin == 1)

X_multi_cond = X_test[cond_idx]
y_multi_cond = y_test[cond_idx]

# 다중 분류 앙상블
w_xgb = 0.4
w_cat = 0.6

multi_proba_xgb = xgb.predict_proba(X_multi_cond)
multi_proba_cat = cat.predict_proba(X_multi_cond)
multi_proba_ens = w_xgb*multi_proba_xgb + w_cat*multi_proba_cat

multi_pred = np.argmax(multi_proba_ens, axis=1)

### 예측 성능 확인

In [11]:
y_pred_full = np.full_like(y_test, fill_value=NEG_LABEL)

if cond_idx.any():
    pred_pos_labels = pos_le.inverse_transform(multi_pred)  # 원래 양성 라벨
    y_pred_full[cond_idx] = pred_pos_labels

    y_true_pass = y_test[cond_idx]                  # 원래 라벨(95 제외)
    stage2_acc_top1_cond = accuracy_score(y_true_pass, pred_pos_labels)
else:
    stage2_acc_top1_cond = np.nan

end2end_acc_top1 = accuracy_score(y_test, y_pred_full)
end2end_f1_micro = f1_score(y_test, y_pred_full, average="micro")
end2end_f1_macro = f1_score(y_test, y_pred_full, average="macro")
end2end_f1_weighted = f1_score(y_test, y_pred_full, average="weighted")


print("Multiclass_acc_top1_conditional:", stage2_acc_top1_cond)
print("end2end_acc_top1:", end2end_acc_top1)
print("end2end_f1_micro:", end2end_f1_micro)
print("end2end_f1_macro:", end2end_f1_macro)
print("end2end_f1_weighted:", end2end_f1_weighted)

Multiclass_acc_top1_conditional: 0.711390284757119
end2end_acc_top1: 0.7695402298850574
end2end_f1_micro: 0.7695402298850574
end2end_f1_macro: 0.7279877035548591
end2end_f1_weighted: 0.7707233266792526


## 확률 기반 예측 성능 확인

In [12]:
# 전체 테스트셋에 대해 양성 클래스 조건부 확률
multi_proba_xgb_all = xgb.predict_proba(X_test)   # shape: (n_samples, C_pos)
multi_proba_cat_all = cat.predict_proba(X_test)   # shape: (n_samples, C_pos)

multi_proba_ens_all = w_xgb * multi_proba_xgb_all + w_cat * multi_proba_cat_all
# 여기가 P(y = c | y != NEG, x)

In [14]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score, average_precision_score

y_true = y_test
n_samples = len(y_true)

# 전체 클래스 (양성 클래스들 + NEG_LABEL)
pos_labels = pos_le.classes_  # LabelEncoder가 본 "원래 양성 라벨"들
all_classes = np.sort(np.concatenate([pos_labels, [NEG_LABEL]]))
num_classes = len(all_classes)

# label -> column index 매핑
class_to_idx = {c: i for i, c in enumerate(all_classes)}

proba_full = np.zeros((n_samples, num_classes), dtype=float)

# 1) NEG_LABEL 확률 = P(y=NEG_LABEL)
neg_col = class_to_idx[NEG_LABEL]
proba_full[:, neg_col] = ens_proba[:, 0]

# 2) 양성 클래스들: P(y != NEG_LABEL) * P(class | y != NEG_LABEL, x)
for j, lbl in enumerate(pos_labels):
    col = class_to_idx[lbl]
    proba_full[:, col] = ens_proba[:, 1] * multi_proba_ens_all[:, j]

In [17]:
# --- multi-class ROC-AUC (OvR, macro/weighted) ---
try:
    end2end_roc_auc_macro = roc_auc_score(
        y_true,
        proba_full,
        multi_class="ovr",
        average="macro",
    )
except ValueError:
    end2end_roc_auc_macro = np.nan
    end2end_roc_auc_weighted = np.nan

# --- multi-class PR-AUC (Average Precision) ---
y_true_bin = label_binarize(y_true, classes=all_classes)
try:
    end2end_pr_auc_macro = average_precision_score(
        y_true_bin,
        proba_full,
        average="macro",
    )
except ValueError:
    end2end_pr_auc_macro = np.nan
    end2end_pr_auc_weighted = np.nan

print("end2end_roc_auc_macro:", end2end_roc_auc_macro)
print("end2end_pr_auc_macro:", end2end_pr_auc_macro)

end2end_roc_auc_macro: 0.9913350089935969
end2end_pr_auc_macro: 0.813885707075891
